# Get cell processing info

In [ ]:
import os
import sys

In [ ]:
sys.path.append(r"../LUCinSA_helpers")
module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)

from mosaic import *
from ras_tools import *
from var_dataframe import *

In [ ]:
out_path = Path('/home/downspout-cel/paraguay_lc/mosaics')
cell_list = '/home/downspout-cel/paraguay_lc/mosaics/lists/Training_cells.csv'
ts_dir = '/home/downspout-cel/paraguay_lc/stac/grids'

In [ ]:
#numrot_wet_mosaic = mosaic_cells(cell_list, ts_dir, 'comp/kndvi', 'numrot_wet_2021', out_path)
numrot_wet_mosaic = '/home/downspout-cel/paraguay_lc/mosaics/numrot_wet_2021_mosaic.tif'
numrot_yr_mosaic = mosaic_cells(cell_list, ts_dir, 'comp/kndvi', 'numrot_yr_2021', out_path)
#numrot_yr_mosaic = '/home/downspout-cel/paraguay_lc/mosaics/numrot_yr_2021_mosaic.tif'

In [ ]:
def get_variables_at_pts_external2(out_dir, ras_in,ptfile):

    ptsdf = pd.read_csv(ptfile, index_col=0)
    ptsgdb = gpd.GeoDataFrame(ptsdf,geometry=gpd.points_from_xy(ptsdf.XCoord,ptsdf.YCoord),crs='epsg:8858')
    #pts4326 = ptsgdb.to_crs({'init': 'epsg:4326'})
    xy = [ptsgdb['geometry'].x, ptsgdb['geometry'].y]
    coords = list(map(list, zip(*xy)))
    
    with rio.open(ras_in, 'r') as comp:
        comp.np = comp.read()
        ptsgdb['numrot_wet'] = [sample[0] for sample in comp.sample(coords)]     

    pd.DataFrame.to_csv(ptsgdb,os.path.join(out_dir,'samp_numrot_wet_2021.csv'), sep=',', index=True)
    return ptsgdb

In [ ]:
out_dir = '/home/downspout-cel/paraguay_lc/vector/tests'
ptfile = '/home/downspout-cel/paraguay_lc/vector/sampleData/SamplePts_Mar2024_ALL.csv'
samp_numrot_wet = get_variables_at_pts_external2(out_dir, numrot_wet_mosaic, ptfile)

In [ ]:
#get_variables_at_pts_external2(out_dir, numrot_yr_mosaic, ptfile)


In [ ]:
samp_numrot_yr = pd.read_csv(os.path.join(out_dir,'samp_numrot_yr_2021.csv'))
samp_numrot_wet = pd.read_csv(os.path.join(out_dir,'samp_numrot_wet_2021.csv'))
#samp_numrot_dry = pd.read_csv(os.path.join(out_dir,'samp_numrot_dry_2021.csv'))
                              
crop_samp_wet = samp_numrot_wet[(samp_numrot_wet['LC_UNQ'] > 30) & (samp_numrot_wet['LC_UNQ'] < 50)]
crop_samp_yr = samp_numrot_yr[(samp_numrot_yr['LC_UNQ'] > 30) & (samp_numrot_yr['LC_UNQ'] < 50)]
crop_samp_wetg = crop_samp_wet[['Class','numrot_wet']].groupby('Class').mean()
crop_samp_yrg = crop_samp_yr[['Class','numrot_yr']].groupby('Class').mean()
crop_samp_numrot = crop_samp_yrg.join(crop_samp_wetg,on='Class',how='left')
print(crop_samp_numrot)